In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import torch

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from surface_vae_trainer import SurfaceVAETrainer
from dupire_pinn_trainer import DupirePINNTrainer
from latent_dimension_assessment import sample_latent_vectors, latent_space_assessment

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
# n_roots = 10
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

prior_covariance_matrix = smoothness_prior.prior_covariance()
prior_eigenvalues = np.sort(np.linalg.eigvalsh(prior_covariance_matrix))[::-1].copy()

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(1000)

# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

In [4]:
hidden_dim = 64
n_layers = 8
batch_size = 100
pde_loss_coefficient = 650.0
maturity_zero_loss_coefficient = 800.0
strike_zero_loss_coefficient = 40.0
strike_infinity_loss_coefficient = 200.0
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 200
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
torch.manual_seed(0)
pinn_trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

pinn_trainer.load_model()

pinn_trainer.dupire_price_prediction_loss(
    torch.tensor(sampled_surface_coefficients, device=device, dtype=torch.float32),
    call_option_dataset["Call Option Price"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
)
print()

PINN Model loaded from models/pinn_model.pth.



In [ ]:
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

# List of candidate latent dimensions
latent_dimensions = [2, 4, 8, 10, 15, 20, 25, 30, 40, 50, 60, 75]

# Initialize dictionaries to store the results for each latent dimension
condition_number_percentiles = {}
lipschitz_constants = {}

# Loop over the candidate latent dimensions
for latent_dim in latent_dimensions:
    # Initialize the VAE trainer for the current latent dimension
    torch.manual_seed(2)
    vae_trainer = SurfaceVAETrainer(
        latent_dim=latent_dim,
        hidden_dim=hidden_dim,  # Keep hidden_dim fixed for all latent dimensions
        n_layers=n_layers,
        data_dim=data_dim,
        latent_diagonal=prior_eigenvalues[:latent_dim],  # Eigenvalues for latent prior
        batch_size=batch_size,
        beta=beta_,
        pre_train_learning_rate=pre_train_learning_rate,
        fine_tune_learning_rate=fine_tune_learning_rate,
        pre_train_epochs=pre_train_epochs,
        fine_tune_epochs=fine_tune_epochs,
        device=device,
    )

    # Train the model using pre_train_with_sampling
    vae_trainer.pre_train_with_sampling(
        smoothness_prior=smoothness_prior,
    )

    # Sample latent vectors and assess the latent space
    # Initialize variables
    all_condition_numbers = []
    max_lipschitz_constant = float('-inf')  # Start with negative infinity for comparison
    # Repeat the process 10 times
    for i in range(100):
        # Sample latent vectors and assess the latent space
        torch.manual_seed(i + 2)  # Update seed for each iteration
        latent_samples_batch = sample_latent_vectors(100, prior_eigenvalues[:latent_dim])
        
        # Assess the latent space for condition numbers and Lipschitz constant
        condition_numbers, lipschitz_constant = latent_space_assessment(latent_samples_batch, vae_trainer, pinn_trainer)
        
        # Extend the condition numbers list with the new condition numbers
        all_condition_numbers.extend(condition_numbers)
        
        # Update the Lipschitz constant by taking the maximum of the current and new value
        max_lipschitz_constant = max(max_lipschitz_constant, lipschitz_constant)

    # Calculate the percentiles of condition numbers
    all_condition_numbers = np.array(all_condition_numbers)
    condition_number_percentiles[latent_dim] = {
        "0.5 Percentile": np.percentile(all_condition_numbers, 50),
        "0.05 Percentile": np.percentile(all_condition_numbers, 5),
        "0.95 Percentile": np.percentile(all_condition_numbers, 95),
    }

    # Record the Lipschitz constant for the current latent dimension
    lipschitz_constants[latent_dim] = max_lipschitz_constant

    # Print progress
    print(f"Latent Dimension: {latent_dim}")
    print(f"Condition Number Percentiles: {condition_number_percentiles[latent_dim]}")
    print(f"Lipschitz Constant: {lipschitz_constants[latent_dim]}\n")

In [30]:
# Prepare data for plotting
latent_dims = list(condition_number_percentiles.keys())
condition_medians = [condition_number_percentiles[ld]["0.5 Percentile"] for ld in latent_dims]
condition_lower = [condition_number_percentiles[ld]["0.05 Percentile"] for ld in latent_dims]
condition_upper = [condition_number_percentiles[ld]["0.95 Percentile"] for ld in latent_dims]
lipschitz_values = [lipschitz_constants[ld] for ld in latent_dims]

# Create the figure
fig = go.Figure()

# Add scatter plot for condition number with error bars
fig.add_trace(go.Scatter(
    x=latent_dims,
    y=condition_medians,
    mode='lines+markers',
    name='Condition Number',
    marker=dict(symbol='circle', size=7.5)
))

# Add scatter plot for Lipschitz constant on secondary y-axis
fig.add_trace(go.Scatter(
    x=latent_dims,
    y=lipschitz_values,
    mode='lines+markers',
    name='Lipschitz Constant',
    yaxis='y2',
    marker=dict(symbol='square', size=7.5)
))

# Update the layout for the double y-axis
fig.update_layout(
    title="Condition Number and Lipschitz Constant vs Latent Dimension",
    xaxis=dict(
        title="Latent Dimension",
        tickvals=latent_dims,
    ),
    yaxis=dict(
        title="Condition Number",
        showgrid=True,
        zeroline=False,
    ),
    yaxis2=dict(
        title="Lipschitz Constant",
        overlaying='y',
        side='right',
        showgrid=False,
        zeroline=False,
    ),
    legend=dict(
        x=0.01, y=0.99,
    ),
    width=900,
    height=900,
)

# Show the figure
fig.show()
fig.write_image('figs/latent_dimension_condition_number_libschitz.png', format='png', scale=4, width=900, height=900)

In [34]:
latent_dim = 40  # Latent dimension
data_dim = 100  # Data dimension of input
hidden_dim = 512
n_layers = 8
latent_diagonal = prior_eigenvalues[:latent_dim]  # Eigenvalues for latent prior
batch_size = 1000  # Batch size for training
beta_ = 1.0  # Beta value for beta-VAE
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4  # Fine-tune learning rate
pre_train_epochs = 600  # Number of pre-train epochs
fine_tune_epochs = 20  # Number of fine-tune epochs
device = "cpu"  # Use CPU as the device

torch.manual_seed(2)
vae_trainer = SurfaceVAETrainer(
    latent_dim=latent_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    data_dim=data_dim,
    latent_diagonal=latent_diagonal,
    batch_size=batch_size,
    beta=beta_,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    device=device,
)

# Train the model using pre_train
vae_trainer.pre_train_with_sampling(
    smoothness_prior=smoothness_prior,
)

vae_trainer.save_model()

loss_history = pd.DataFrame(vae_trainer.pre_train_loss_history)

# Create a subplot figure with 1x2 grid for individual losses, and a second row spanning the entire width for total loss
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Reconstruction Loss", "KL Loss", "Total Loss"),
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'colspan': 2, 'type': 'scatter'}, None]],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add traces for individual losses
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Reconstruction Loss"], mode="lines", name="Reconstruction Loss"), row=1, col=1)
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["KL Loss"], mode="lines", name="KL Loss"), row=1, col=2)

# Add a trace for the total loss spanning the entire second row
fig.add_trace(go.Scatter(x=loss_history.index, y=loss_history["Total Loss"], mode="lines", name="Total Loss"), row=2, col=1)

# Update the layout to include 'Iterations' as the x-axis name for each subplot
fig.update_xaxes(title_text="Iterations", row=1, col=1)
fig.update_xaxes(title_text="Iterations", row=1, col=2)
fig.update_xaxes(title_text="Iterations", row=2, col=1)  # The third row spans two columns

fig.update_yaxes(type="log", row=1, col=1)
fig.update_yaxes(type="log", row=1, col=2)
fig.update_yaxes(type="log", row=2, col=1)  # The third row spans two columns

# Update the layout
fig.update_layout(height=900, width=900, title_text="Beta-VAE Training Losses", showlegend=False)

# Show the plot
fig.show()
fig.write_image('figs/vae_training_loss_history.png', format='png', scale=4, width=900, height=900)

100%|██████████| 600/600 [00:29<00:00, 20.09it/s]

VAE Model saved to models/vae_model.pth.
